In [0]:
!pip install tensorflow==1.14.0

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

from PIL import Image

In [0]:
from google.colab import drive #STU NAPERVILLE ACCOUNTS DON'T WORK - USE YOUR PERSONAL GOOGLE ACCOUNTS
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, brightness_range=(-1, 1))
train_generator = train_datagen.flow_from_directory(
        "/content/drive/My Drive/ROADREADER DATASETS/training",
        target_size=(300, 400),
        batch_size=32, 
        color_mode="rgb",
        class_mode="sparse",
        shuffle=True)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, brightness_range=(-1, 1))
test_generator = test_datagen.flow_from_directory(
        "/content/drive/My Drive/ROADREADER DATASETS/tests",
        target_size=(300, 400),
        batch_size=1, 
        color_mode="rgb",
        class_mode=None,
        shuffle=False)

Found 1196 images belonging to 2 classes.
Found 204 images belonging to 2 classes.


In [0]:
model = keras.Sequential([
  layers.Conv2D(14, 3, activation='relu', input_shape=(300,400,3)),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(28, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(42, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Flatten(),
  layers.Dense(20000, activation='relu'),
  layers.Dense(200, activation='relu'),
  layers.Dense(2, activation='softmax'), #classification layer; 0 for none 1 for stop
])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 398, 14)      392       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 199, 14)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 197, 28)      3556      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 99, 28)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 97, 42)        10626     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 49, 42)        0         
____

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
#model.fit(x=train_images,y=train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=128)

model.fit_generator(
        train_generator,
        steps_per_epoch=128,
        epochs=10,
        validation_data=test_generator,
        validation_steps=32,
        workers=1,
        use_multiprocessing=False,
        max_queue_size=10)

'''
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)
                    '''

model.save_weights('/content/model.h5')

NameError: ignored